In [1]:
import anatomist.api as ana
from soma.qt_gui.qtThread import QtThreadCall
from soma.qt_gui.qt_backend import Qt

from soma import aims
import numpy as np
import pandas as pd
import os

from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm



/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
a = ana.Anatomist()

create qapp
global modules: /casa/host/build/share/anatomist-5.2/python_plugins
home   modules: /casa/home/.anatomist/python_plugins
done
Starting Anatomist.....
config file : /casa/home/.anatomist/config/settings.cfg
PyAnatomist Module present
PythonLauncher::runModules()
loading module meshsplit
loading module palettecontrols
loading module volumepalettes


existing QApplication: 0
QStandardPaths: XDG_RUNTIME_DIR not set, defaulting to '/tmp/runtime-jc225751'


loading module save_resampled
loading module profilewindow
loading module statsplotwindow
loading module anacontrolmenu
loading module infowindow
loading module foldsplit
loading module simple_controls
loading module histogram
loading module valuesplotwindow
loading module ana_image_math
loading module modelGraphs
loading module paletteViewer
loading module bsa_proba
loading module gradientpalette
loading module selection
loading module gltf_io
all python modules loaded
Anatomist started.


### Variable definitions

In [3]:
embeddings_file = "/neurospin/dico/data/deep_folding/current/models/Champollion_V0/FPO-SCu-ScCal_left/20-03-37_0/ukb_random_embeddings/full_embeddings.csv"
participants_file = "/neurospin/dico/data/deep_folding/current/datasets/UkBioBank/participants.csv"

In [4]:
label = "Sex"

In [5]:
side = "L" # "R" or "L"
region = "F.P.O.-S.Cu.-Sc.Cal." # "S.C.-sylv.", "ORBITAL", "CINGULATE", "SC-sylv", "F.I.P."
database='ukb'

In [6]:
def closest_point(point, points):
    """ Find closest point from a list of points. """
    return np.argsort((points-point)**2)[:10]

# Building predictors

In [7]:
# Builds participants containing 10 and 90-quantile of label_before
participants = pd.read_csv(participants_file, index_col=0)

q_80 = participants["BrainVolumeFreeSurfer_mm3"].quantile(0.8)
participants = participants[participants["BrainVolumeFreeSurfer_mm3"]>=q_80]

participants = participants[[label, "BrainVolumeFreeSurfer_mm3"]]

females = participants[participants[label]==0].copy()
males = participants[participants[label]==1].copy()

females = participants[participants.Sex==0].copy()
males = participants[participants.Sex==1].copy()

males["chosen"] = 0
for one_female in females.iloc[:,1]:
    chosen_rows = closest_point(one_female, males.iloc[:,1])
    for chosen_row in chosen_rows:
        if males.iloc[chosen_row, 2] == 0:
            males.iloc[chosen_row, 2] = 1
            break

participants = pd.concat([females, males[males.chosen==1][["Sex", "BrainVolumeFreeSurfer_mm3"]]])
participants = participants[[label]].dropna()
participants

,Sex
participant_id,
sub-5458483,0
sub-2011528,0
sub-4205268,0
sub-2843269,0
sub-3231417,0
...,...
sub-2641491,1
sub-5182900,1
sub-1569024,1


In [8]:
embeddings = pd.read_csv(f"{embeddings_file}", index_col=0)

In [9]:
merged = participants[[label]].merge(embeddings, left_index=True, right_index=True)

# Classifies
X = merged.drop([label], axis=1)
Y = merged[[label]].astype(float)

# Standard scaler
scaler = StandardScaler()
X[X.columns] = scaler.fit_transform(X)
df2 = X.copy()

# Makes OLS
df2 = sm.add_constant(df2)
model = sm.OLS(Y[label], df2)
results = model.fit()

p = results.params

# Gets predictor
df = Y.copy()
df["predicted"] = df2.dot(p)

results.fvalue

2.1536025689757112

In [10]:
df

,Sex,predicted
sub-2011528,0.0,0.500998
sub-2843269,0.0,0.296675
sub-5108916,0.0,0.167509
sub-4993669,0.0,0.369938
sub-4074924,0.0,0.600783
...,...,...
sub-3579605,1.0,1.238913
sub-3448516,1.0,1.001406
sub-2047066,1.0,1.459223
sub-2641491,1.0,0.725740


### Function definitions

In [11]:
def to_bucket(obj):
    if obj.type() == obj.BUCKET:
        return obj
    avol = a.toAimsObject(obj)
    c = aims.Converter(intype=avol, outtype=aims.BucketMap_VOID)
    abck = c(avol)
    bck = a.toAObject(abck)
    bck.releaseAppRef()
    return bck

def build_gradient(pal):
    gw = ana.cpp.GradientWidget(None, 'gradientwidget', pal.header()['palette_gradients'])
    gw.setHasAlpha(True)
    nc = pal.shape[0]
    rgbp = gw.fillGradient(nc, True)
    rgb = rgbp.data()
    npal = pal.np['v']
    pb = np.frombuffer(rgb, dtype=np.uint8).reshape((nc, 4))
    npal[:, 0, 0, 0, :] = pb
    npal[:, 0, 0, 0, :3] = npal[:, 0, 0, 0, :3][:, ::-1]  # BGRA -> RGBA
    pal.update()

def buckets_average(subject_id_list, dataset_name_list):
    dic_vol = {}
    dim = 0
    rep = 0
    if len(subject_id_list) == 0:
        return False
    while dim == 0 and rep < len(subject_id_list):
        if dataset_name_list[rep].lower() in ['ukb', 'ukbiobank', 'projected_ukb']:
            dataset = 'UkBioBank'
        elif dataset_name_list[rep].lower() in ['hcp']:
            dataset = 'hcp'
        mm_skeleton_path = f"/neurospin/dico/data/deep_folding/current/datasets/{dataset}/crops/2mm/{region}/mask/{side}crops"
        if os.path.isfile(f'{mm_skeleton_path}/{subject_id_list[rep]}_cropped_skeleton.nii.gz'):
            sum_vol = aims.read(f'{mm_skeleton_path}/{subject_id_list[rep]}_cropped_skeleton.nii.gz').astype(float)
            dim = sum_vol.shape
            sum_vol.fill(0)
        else: 
            print(f'FileNotFound {mm_skeleton_path}/{subject_id_list[rep]}_cropped_skeleton.nii.gz')
            #raise FileNotFoundError(f'{mm_skeleton_path}/{subject_id_list[0]}_cropped_skeleton.nii.gz')
        rep += 1

    for subject_id, dataset in zip(subject_id_list,dataset_name_list):
        if dataset.lower() in ['ukb', 'ukbiobank',  'projected_ukb']:
            dataset = 'UkBioBank'
        elif dataset.lower() == 'hcp':
            dataset = 'hcp'
            
        mm_skeleton_path = f"/neurospin/dico/data/deep_folding/current/datasets/{dataset}/crops/2mm/{region}/mask/{side}crops"

        if os.path.isfile(f'{mm_skeleton_path}/{subject_id}_cropped_skeleton.nii.gz'):
            vol = aims.read(f'{mm_skeleton_path}/{subject_id}_cropped_skeleton.nii.gz')
            # compare the dim with the first file dim

            if vol.np.shape != dim:
                raise ValueError(f"{subject_id_list[0]} and {subject_id} must have the same dim")

                
            # to have a binary 3D structure
            dic_vol[subject_id] = (vol.np > 0).astype(int)
            sum_vol.np[:] += (vol.np > 0).astype(int) 
        else: 
            print(f'FileNotFound {mm_skeleton_path}/{subject_id}_cropped_skeleton.nii.gz')
            #raise FileNotFoundError(f'{mm_skeleton_path}/{subject_id}_cropped_skeleton.nii.gz')

    sum_vol = sum_vol / len(subject_id_list)
    print(sum_vol, sum_vol.shape)
    return sum_vol

In [12]:
def visualize_averages_along_parameter(df, column_name, database, nb_subjects_per_average=200, nb_columns=2, nb_lines=1):
    # anatomist objects
    global _block
    global _average_dic
    global dic_packages
    nb_windows = nb_columns * nb_lines
    _average_dic = {}
    step = nb_subjects_per_average # number of subjects on which average is done
    # Creates the block if it has not been created
    # try:
    #     _block
    # except NameError:
    _block = a.createWindowsBlock(nb_columns)

    # Order according to column_name
    df = df.sort_values(column_name)
    predict_proba = df[[column_name]]

    # Creates dictionary of subjects to average
    dic_packages = {}
    for i in range(0,len(predict_proba),step):
        list_idx = (predict_proba.index[i:i+step].to_numpy())
        dic_packages[i//step] = list_idx
    
    # Ensures that last list contains the last step subjects
    list_idx = (predict_proba.index[-step:].to_numpy())
    dic_packages[i//step] = list_idx
    
    list_database = [database for i in range(step)]
    n_pack = len(dic_packages)

    # Loop of display averages
    list_pack = [int(np.ceil(i*n_pack/float(nb_windows))) for i in range(0,nb_windows)]
    for i in list_pack:
        sum_vol = buckets_average(dic_packages[i], list_database)
        _average_dic[f'a_sum_vol{i}'] = a.toAObject(sum_vol)
        _average_dic[f'a_sum_vol{i}'].setPalette(minVal=0, absoluteMode=True)
        #wsum = a.createWindow('Sagittal', block=block)
        #wsum.addObjects(a_sum_vol)
        _average_dic[f'rvol{i}'] = a.fusionObjects(
            objects=[_average_dic[f'a_sum_vol{i}']],
            method='VolumeRenderingFusionMethod')
        _average_dic[f'rvol{i}'].releaseAppRef()
        # custom palette
        n = len(dic_packages[i])
        pal = a.createPalette('VR-palette')
        pal.header()['palette_gradients'] = '0;0.459574;0.497872;0.910638;1;1#0;0;0.52766;0.417021;1;1#0;0.7;1;0#0;0;0.0297872;0.00851064;0.72766;0.178723;0.957447;0.808511;1;1'
        #f'0;0.244444;0.5;1;1;1#0;0;0.535897;0.222222;1;1#0;0.7;1;0#0;0;{0.5/n};0;1;1'
        build_gradient(pal)
        _average_dic[f'rvol{i}'].setPalette('VR-palette', minVal=0.05, absoluteMode=True)
        pal2 = a.createPalette('slice-palette')
        pal2.header()['palette_gradients'] = '0;0.459574;0.497872;0.910638;1;1#0;0;0.52766;0.417021;1;1#0;0.7;1;0#0;0;0.0297872;0.00851064;0.72766;0.178723;0.957447;0.808511;1;1'
        #f'0;0.244444;0.5;1;1;1#0;0;0.535897;0.222222;1;1#0;0.7;1;0#0;0;{0.3/n};0;{0.7/n};1;1;1'
        build_gradient(pal2)
        _average_dic[f'a_sum_vol{i}'].setPalette('slice-palette')
        # rvol.palette().fill()
        _average_dic[f'wvr{i}'] = a.createWindow('3D', block=_block)
        _average_dic[f'wvr{i}'].addObjects(_average_dic[f'rvol{i}'])

### Visualization

In [13]:
# block = a.createWindowsBlock(10)
visualize_averages_along_parameter(df, "predicted", database, nb_subjects_per_average=200)

<soma.aims.Volume_DOUBLE object at 0x7c2ed5731d80> (25, 42, 54, 1)
Multitexturing present
function glActiveTexture found.
function glClientActiveTexture found.
function glBlendEquation found.
function glTexImage3D found.
function glMultiTexCoord3f found.
function glBindFramebuffer found.
function glBindRenderbuffer found.
function glFramebufferTexture2D found.
function glGenFramebuffers found.
function glGenRenderbuffers found.
function glFramebufferRenderbuffer found.
function glRenderbufferStorage found.
function glCheckFramebufferStatus found.
function glDeleteRenderbuffers found.
function glDeleteFramebuffers found.
Number of texture units: 4
function glUniform1f found.
function glUniform1i found.
function glUniform4fv found.
function glGetUniformLocation found.
function glMultTransposeMatrixf found.
function glAttachShader found.
function glDetachShader found.
function glCompileShader found.
function glCreateProgram found.
function glCreateShader found.
function glDeleteProgram fo

In [14]:
type(_block)

anatomist.direct.api.Anatomist.AWindowsBlock

In [15]:
df.sort_values("predicted")

,Sex,predicted
sub-1742429,0.0,-0.334446
sub-5326062,0.0,-0.274432
sub-3038546,0.0,-0.267272
sub-2005036,0.0,-0.237447
sub-4214216,0.0,-0.181294
...,...,...
sub-4763788,1.0,1.433408
sub-1050729,1.0,1.452829
sub-2047066,1.0,1.459223
sub-1376118,1.0,1.479974


# Displaying subjects

In [16]:
# list_subjects = list_smallest
# block = a.createWindowsBlock(5) # 5 columns
# dic_windows = {}
# for subject in list_subjects:
#     path_to_t1mri = f'/neurospin/dico/data/bv_databases/human/not_labeled/hcp/hcp/{subject}/t1mri/BL'
#     dic_windows[f'w{subject}'] = a.createWindow("3D", block=block)
#     dic_windows[f'white_{subject}'] = a.loadObject(f'{path_to_t1mri}/default_analysis/segmentation/mesh/{subject}_{side}white.gii')
#     dic_windows[f'white_{subject}'].loadReferentialFromHeader()
#     dic_windows[f'sulci_{subject}'] = a.loadObject(f'{path_to_t1mri}/default_analysis/folds/3.1/{side}{subject}.arg')
#     dic_windows[f'sulci_{subject}'].loadReferentialFromHeader()
#     dic_windows[f'w{subject}'].addObjects([dic_windows[f'white_{subject}'], dic_windows[f'sulci_{subject}']])

In [17]:
df.sort_values("predicted")[::10]

,Sex,predicted
sub-1742429,0.0,-0.334446
sub-5766961,0.0,-0.138263
sub-5713234,0.0,-0.064428
sub-4852156,0.0,0.009732
sub-3113494,0.0,0.059271
sub-5462352,0.0,0.077101
sub-1222541,0.0,0.110595
sub-3469847,0.0,0.132565
sub-1778458,0.0,0.153584
sub-4499906,0.0,0.185258


In [18]:
df_concat = pd.concat([df.sort_values("predicted")[::10][:5], df.sort_values("predicted")[::10][-5:]])

In [19]:
df_concat

,Sex,predicted
sub-1742429,0.0,-0.334446
sub-5766961,0.0,-0.138263
sub-5713234,0.0,-0.064428
sub-4852156,0.0,0.009732
sub-3113494,0.0,0.059271
sub-1295988,1.0,1.126027
sub-3853971,1.0,1.163019
sub-1204990,1.0,1.204456
sub-5016057,1.0,1.281388
sub-5438860,1.0,1.395432


In [20]:
# visualize_averages_along_parameter(df_concat, "predicted", database, nb_subjects_per_average=1, nb_columns=5, nb_lines=2)

QLayout: Attempting to add QLayout "" to QWidget "", which already has a layout


no position could be read at 351, 225
